# Matrix Condition Number and Roundoff Error

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%204/Matrix_Condition_Number_Demo.ipynb)

## Introduction

In this notebook, we'll explore the concept of **matrix condition number** and how it affects the numerical solution of linear systems of equations. We'll specifically focus on:

1. What makes a matrix "ill-conditioned"
2. How roundoff error accumulates in ill-conditioned systems
3. How to check the condition number of a matrix
4. Basic techniques to improve numerical stability

In [ ]:
# Import the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.linalg import cond, solve, norm, inv
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Configure plot settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.5)

# Set precision for display
np.set_printoptions(precision=8, suppress=True)

## What is the Condition Number?

The **condition number** of a matrix measures how sensitive the solution of a linear system is to small changes or errors in the input data. 

For a matrix $A$, the condition number is defined as:

$$\kappa(A) = ||A|| \cdot ||A^{-1}||$$

where $||\cdot||$ represents a matrix norm (commonly the 2-norm).

- A matrix with a **low condition number** (close to 1) is **well-conditioned**. Small changes in the input result in small changes in the output.
- A matrix with a **high condition number** is **ill-conditioned**. Small changes in the input can cause large changes in the output.
- A matrix with an **infinite condition number** is **singular** (not invertible).

## Example 1: Well-Conditioned Matrix

Let's start with a well-conditioned matrix and see how it behaves when solving a linear system.

In [ ]:
# Create a well-conditioned matrix (a simple diagonal matrix)
A_well = np.array([
    [4.0, 0.0],
    [0.0, 5.0]
])

# True solution we want to recover
x_true = np.array([2.0, 3.0])

# Calculate the right-hand side
b = A_well @ x_true

print("Matrix A (well-conditioned):")
print(A_well)
print("\nRight-hand side b:")
print(b)

# Calculate the condition number
cond_number = cond(A_well)
print(f"\nCondition number: {cond_number:.4f}")

# Solve the system
x_computed = solve(A_well, b)
print("\nComputed solution:")
print(x_computed)

# Calculate the error
error = norm(x_true - x_computed) / norm(x_true)
print(f"\nRelative error: {error:.8e}")

Now, let's introduce some small perturbation to the right-hand side and see how it affects the solution:

In [ ]:
# Add a small perturbation to b (simulating measurement error or roundoff)
delta = 1e-10
b_perturbed = b + delta * np.random.randn(2)

# Solve with the perturbed right-hand side
x_perturbed = solve(A_well, b_perturbed)

# Calculate the relative change in the solution
input_change = norm(b - b_perturbed) / norm(b)
output_change = norm(x_computed - x_perturbed) / norm(x_computed)

print(f"Relative change in input (b): {input_change:.8e}")
print(f"Relative change in output (x): {output_change:.8e}")
print(f"Ratio of output change to input change: {output_change/input_change:.4f}")

For a well-conditioned matrix, the relative change in the solution is similar in magnitude to the relative change in the input data. The ratio is close to the condition number.

## Example 2: Moderately Ill-Conditioned Matrix

Now, let's examine a moderately ill-conditioned matrix and observe how it amplifies errors.

In [ ]:
# Create a Hilbert matrix - a classic example of an ill-conditioned matrix
# The Hilbert matrix H has elements H[i,j] = 1/(i+j+1)
def hilbert_matrix(n):
    H = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            H[i, j] = 1.0 / (i + j + 1)
    return H

# Create a 7x7 Hilbert matrix (moderate size)
n = 7
A_ill = hilbert_matrix(n)

# Create a simple true solution - all ones
x_true_ill = np.ones(n)

# Calculate right-hand side
b_ill = A_ill @ x_true_ill

print("Hilbert matrix (ill-conditioned, showing first 5x5 corner):")
print(A_ill[:5, :5])

# Calculate the condition number
cond_number_ill = cond(A_ill)
print(f"\nCondition number: {cond_number_ill:.4e}")

# Solve the system using double precision
x_computed_ill = solve(A_ill, b_ill)

print("\nTrue solution:")
print(x_true_ill)

print("\nComputed solution:")
print(x_computed_ill)

# Calculate the element-wise absolute errors
abs_errors = np.abs(x_true_ill - x_computed_ill)
print("\nAbsolute errors:")
print(abs_errors)

# Calculate the relative error
error_ill = norm(x_true_ill - x_computed_ill) / norm(x_true_ill)
print(f"\nRelative error: {error_ill:.8e}")

Let's add a small perturbation and observe its effect:

In [ ]:
# Add a small perturbation to b_ill
delta_ill = 1e-8  # Small but not too small perturbation
np.random.seed(123)  # Set seed for reproducibility
perturbation = delta_ill * np.random.randn(n)
b_perturbed_ill = b_ill + perturbation

# Solve with the perturbed right-hand side
x_perturbed_ill = solve(A_ill, b_perturbed_ill)

# Calculate the relative changes
input_change_ill = norm(b_ill - b_perturbed_ill) / norm(b_ill)
output_change_ill = norm(x_computed_ill - x_perturbed_ill) / norm(x_computed_ill)

print(f"Relative change in input (b): {input_change_ill:.8e}")
print(f"Relative change in output (x): {output_change_ill:.8e}")
print(f"Ratio of output change to input change: {output_change_ill/input_change_ill:.4e}")

# Compare this ratio to the condition number
print(f"Condition number: {cond_number_ill:.4e}")

# Calculate errors of perturbed solution vs true solution
perturbed_error = norm(x_true_ill - x_perturbed_ill) / norm(x_true_ill)
print(f"\nRelative error of perturbed solution: {perturbed_error:.8e}")

Let's create a better visualization that clearly shows the effects without one solution overwhelming the others:

In [ ]:
# Create a dataframe for comparison
df_comparison = pd.DataFrame({
    'Index': range(n),
    'True Solution': x_true_ill,
    'Computed Solution': x_computed_ill,
    'Perturbed Solution': x_perturbed_ill,
    'Error in Computed': abs_errors,
    'Error in Perturbed': np.abs(x_true_ill - x_perturbed_ill),
    'Difference Between Solutions': np.abs(x_computed_ill - x_perturbed_ill)
})

print("Comparison of Solutions and Errors:")
df_comparison

In [ ]:
# Create improved visualizations
plt.figure(figsize=(14, 10))

# Plot 1: Solutions on separate plots to avoid scale issues
plt.subplot(2, 2, 1)
plt.bar(range(n), x_true_ill, width=0.6, color='green')
plt.axhline(y=1, color='black', linestyle='--', alpha=0.3)
plt.title('True Solution')
plt.xlabel('Index')
plt.ylabel('Value')
plt.grid(True, alpha=0.3)
plt.xticks(range(n))

plt.subplot(2, 2, 2)
plt.bar(range(n), x_computed_ill, width=0.6, color='blue')
plt.axhline(y=1, color='black', linestyle='--', alpha=0.3)
plt.title('Computed Solution')
plt.xlabel('Index')
plt.ylabel('Value')
plt.grid(True, alpha=0.3)
plt.xticks(range(n))

# Plot 2: Side-by-side comparison of true vs. computed (limited scale)
plt.subplot(2, 2, 3)
indices = np.arange(n)
width = 0.35
plt.bar(indices - width/2, x_true_ill, width, label='True', color='green')
plt.bar(indices + width/2, x_computed_ill, width, label='Computed', color='blue')
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('True vs. Computed Solution')
plt.xticks(indices)
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Logarithmic plot of errors - modified to fix purple line issue
plt.subplot(2, 2, 4)
error_computed = np.abs(x_true_ill - x_computed_ill)
diff_solutions = np.abs(x_computed_ill - x_perturbed_ill)
plt.semilogy(indices, error_computed, 'o-', label='Error in Computed Solution', color='blue')
plt.semilogy(indices, diff_solutions, '^-', label='Difference Between Solutions', color='purple', linewidth=2)
plt.xlabel('Index')
plt.ylabel('Absolute Error (log scale)')
plt.title('Errors (Logarithmic Scale)')
plt.xticks(indices)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Now let's visualize the amplification of errors due to perturbation in a more controlled way:

In [ ]:
# Create a sequence of perturbations with increasing magnitude
perturbation_factors = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6]
solutions = []
output_changes = []
input_changes = []
amplification_factors = []

# Generate solutions for each perturbation level
np.random.seed(42)  # Ensure reproducibility
perturbation_direction = np.random.randn(n)  # Fixed direction, varying magnitude
perturbation_direction = perturbation_direction / norm(perturbation_direction)  # Normalize

for factor in perturbation_factors:
    # Create perturbed right-hand side
    b_pert = b_ill + factor * perturbation_direction
    
    # Solve the perturbed system
    x_pert = solve(A_ill, b_pert)
    solutions.append(x_pert)
    
    # Calculate changes
    input_change = norm(b_ill - b_pert) / norm(b_ill)
    output_change = norm(x_computed_ill - x_pert) / norm(x_computed_ill)
    
    input_changes.append(input_change)
    output_changes.append(output_change)
    amplification_factors.append(output_change / input_change)

# Visualize how the solution changes with increasing perturbation
plt.figure(figsize=(14, 10))

# Plot of solution changes
plt.subplot(2, 1, 1)
colors = ['skyblue', 'royalblue', 'blue', 'darkblue', 'navy']
for i, (factor, sol) in enumerate(zip(perturbation_factors, solutions)):
    plt.plot(range(n), sol, 'o-', label=f'Perturbation {factor:.0e}', color=colors[i], linewidth=2)
plt.plot(range(n), x_true_ill, 'k--', label='True Solution', linewidth=2)
plt.plot(range(n), x_computed_ill, 'k-', label='Original Computed', linewidth=2)
plt.xlabel('Index')
plt.ylabel('Solution Value')
plt.title('Solution Variation with Increasing Perturbation')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot of amplification factors
plt.subplot(2, 1, 2)
plt.loglog(perturbation_factors, amplification_factors, 'o-', linewidth=2, color='darkblue')
plt.axhline(y=cond_number_ill, color='r', linestyle='--', label=f'Condition Number: {cond_number_ill:.2e}')
plt.xlabel('Perturbation Magnitude')
plt.ylabel('Error Amplification Factor')
plt.title('Error Amplification vs. Perturbation Magnitude (Log-Log Scale)')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

# Create a table of results
result_table = pd.DataFrame({
    'Perturbation': perturbation_factors,
    'Input Change': input_changes,
    'Output Change': output_changes,
    'Amplification Factor': amplification_factors
})
print("Error Amplification Data:")
result_table